In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 
import warnings
from tqdm import tqdm
from datetime import datetime
import json
import plotly.express as px
from ast import literal_eval

plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = [20, 8]


In [ ]:
train = pd.read_csv('../data/train.csv')
train.info()

In [ ]:
test = pd.read_csv('../data/test.csv')
test.info()

#### Thoughts on variables:
- A big **budget** does not necessarily leads to a big revenue, money needs to be spent on the right places.
- a **language** might not influence the revenue, but the **number of spoken languages** might do. (check if the number of spoken languages influences the revenue)
- **popularity** is a floating varible; it does not seem to influence the revenue so I decided to drop it
- **Crew** is definitely an very important factor in predicting the box office. I can import some external data to measure the influence of an individual actor/producer, like the number of followers on Twitter.
- Does **ratings** of the movies influence revenues?


#### Unused columns for modeling:
- id
- imdb_id
- original_title
- (overview0)
- popularity (what is it exactly?)
- poster_path

### Thoughts about external datasets:
- Box offices of movies increasingly rely on their performance in foreign markets; is it possible to integrate the information of foreign market distribution?
- Missing budget/revenue data, it can be recovered(scrapped from imdbpro)
- 

In [ ]:
# drop these columns, for now
train = train.drop(columns=['id', 'popularity', 'homepage', 'poster_path', 'status', 'original_title', 'overview', 'tagline'])
test = test.drop(columns=['id', 'popularity', 'homepage', 'poster_path', 'status', 'original_title', 'overview', 'tagline'])

# save imdb_id, title for now

In [ ]:
train['budget'] = train['budget'].replace(0, np.nan)
test['budget'] = test['budget'].replace(0, np.nan)
train['revenue'] = train['revenue'].replace(0, np.nan)

## EDA

In [ ]:
train.head(100)

In [ ]:
train.describe()

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

In [ ]:
# introduce new variable: earn_money, try to figure out what cause a movie to lose revenue
train['earn_money'] = train['budget'] < train['revenue']

### Numerical Variables:

In [ ]:
train[['budget', 'runtime', 'revenue']].corr()

We can see that the correlation between revenue and budget is most obvious.

#### Budget:

In [ ]:
sns.jointplot(x=train['budget'], y=train['revenue'], height=9, ratio=6)

In [ ]:
sns.jointplot(x=train['runtime'], y=train['revenue'], height=9, ratio=6)

### Date_released:
I found date_released to be an really interesting variable. It can be a lens by which we can discover the trends and patterns of the movie market. Before digging into the dataset, I have these questions in mind:
- How do revenues fluctuate along these years?
- How do revenues fluctuate between different months of each year? 
- Do lengths of films change?  

In [ ]:
#Since only last two digits of year are provided, this is the correct way of getting the year.
train[['release_month','release_day','release_year']]=train['release_date'].str.split('/',expand=True).replace(np.nan, -1).astype(int)
# Some rows have 4 digits of year instead of 2, that's why I am applying (train['release_year'] < 100) this condition
train.loc[ (train['release_year'] <= 19) & (train['release_year'] < 100), "release_year"] += 2000
train.loc[ (train['release_year'] > 19)  & (train['release_year'] < 100), "release_year"] += 1900

releaseDate = pd.to_datetime(train['release_date']) 
train['release_dayofweek'] = releaseDate.dt.dayofweek
train['release_quarter'] = releaseDate.dt.quarter

**A General View:**

In [ ]:
plt.rcParams['figure.figsize'] = [20, 8]

avg_rev_by_year = train.groupby('release_year')['revenue'].sum()
avg_rev_by_year_plot = sns.barplot(x=avg_rev_by_year.index, y=avg_rev_by_year.values)
avg_rev_by_year_plot.set_xticklabels(labels=avg_rev_by_year.index, rotation=90)
avg_rev_by_year_plot.set(xlabel='Year', ylabel='Average Revenue', title='Average Revenue by Year, 1921-2017')
plt.show()

**We can see that there can be huge difference between average revenues between the years. Interestingly, if you take a look at the history of Hollywood,(TODO)**

In [ ]:
num_by_year = train.groupby('release_year')['imdb_id'].count()
num_by_year_plot = sns.barplot(x=num_by_year.index, y=num_by_year.values)
num_by_year_plot.set_xticklabels(labels=num_by_year.index, rotation=90)
num_by_year_plot.set(xlabel='Year', ylabel='Number of Movies Releaed', title='Number of Movies Released by Year, From 1921 To 2017')
plt.show()

#### Total number of movies released per year increased along the years obviously.

### Runtime:

In [ ]:
avg_runtime_by_year = train.groupby('release_year')['runtime'].mean()
avg_runtime_by_year_plot = sns.barplot(x=avg_runtime_by_year.index, y =avg_runtime_by_year.values)
avg_runtime_by_year_plot.set_xticklabels(labels=avg_runtime_by_year.index, rotation=90)
avg_runtime_by_year_plot.set(xlabel='Year', ylabel='Average Runtime', title='Average Runtime of Movies, 1921-2017')
plt.show()

**There no clear trend for the length of movies.**

### Revenue by Month / Number of Movies Released by Month: (Release_Date)

In [ ]:
avg_rev_by_month = train.groupby('release_month')['revenue'].mean()
avg_rev_by_month_plot = sns.barplot(x= avg_rev_by_month.index, y=avg_rev_by_month.values)
avg_rev_by_month_plot.set(xlabel='Month', ylabel='Average Revenue', title='Average Revenues by Month, 1921-2017')

#### By observing the average monthly revenues through 1921 to 2017, a trend can be found that average revenues were lowest between Jan to Feb and again Aug to Sept.

In [ ]:
df = train[train['release_year'] > 2010]
avg_rev_by_year_and_month = df.groupby(['release_year', 'release_month'], as_index=False)['revenue'].mean()
avg_rev_by_year_and_month_plot = sns.lineplot(data=avg_rev_by_year_and_month, x='release_month', y='revenue', hue = 'release_year', legend = 'full', palette='Set1')
avg_rev_by_year_and_month_plot.set(xlabel='Month', ylabel='Revenue', title='Monthly Average Revenues by Month, 2011-2017')
avg_rev_by_year_and_month_plot

**The same trend can be found when taking a closer look at monthly average revenues between 2010-2017.**

In [ ]:
num_of_movies_by_year_and_month = df.groupby(['release_year', 'release_month'], as_index=False)['imdb_id'].count()
num_of_movies_by_year_and_month_plot = sns.lineplot(data=num_of_movies_by_year_and_month, x='release_month', y='imdb_id', hue='release_year', legend='full', palette='Set1')
num_of_movies_by_year_and_month_plot.set(xlabel='Month', ylabel='Number of Movies released', title='Monthly Number of Released Movies, 2011-2017')
plt.show()

**By contrast, August to October see the most number of movies released when we look at the data from 2011-2017. (The same pattern persists for previous years also.)**


**A closer look at this trend is provided below, where we can compare the average revenues with number of releases each year:**

In [ ]:
def display_monthly_avg_rev_by_year(year):
    df = train[train['release_year'] == year]
    table = df.groupby('release_month')['revenue'].mean()
    avg_rev_by_month_plot = sns.barplot(x=table.index, y=table.values)
    avg_rev_by_month_plot.set(xlabel='Month', ylabel='Average Revenue', title='Average Revenues by Month in '+ str(year))
    plt.show()
    
def display_monthly_num_of_movies_by_year(year):
    df = train[train['release_year'] == year]
    table = df.groupby('release_month', as_index=False)['imdb_id'].count()
    plot = sns.barplot(data=table, x='release_month', y='imdb_id')
    plot.set(xlabel='Month', ylabel='Number of Movies released', title='Number of Released Movies by Month in ' + str(year))
    plt.show()

def compare_num_with_rev_by_year(year):
    display_monthly_avg_rev_by_year(year)
    display_monthly_num_of_movies_by_year(year)


In [ ]:
compare_num_with_rev_by_year(2010)

A trend can be identified by comparing monthly number of movies released and monthly average revenues: they seem to be inversely correlated.

### Genre:

In [ ]:
def get_dictionary(s):
    try:
        d = eval(s)
    except:
        d = {}
    return d
train = train
train['genres'] = train['genres'].map(lambda x: sorted([d['name'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str, x)))

genres = train.genres.str.get_dummies(sep=',')
# # train = pd.concat([train, genres], axis=1, sort=False)
# print("Action:", train[train['Action'] == 1].shape[0])
# print("Adventure:", train[train.Adventure == 1].shape[0])
# print("Animation:", train[train.Animation == 1].shape[0])
# print("Comedy:", train[train.Comedy == 1].shape[0])
# print("Crime:", train[train.Crime == 1].shape[0])
# print("Documentary:", train[train.Documentary == 1].shape[0])
# print("Drama:", train[train.Drama == 1].shape[0])
# print("Family:", train[train.Family == 1].shape[0])
# print("Fantasy:", train[train.Fantasy == 1].shape[0])
# print("Foreign:", train[train.Foreign == 1].shape[0])
# print("History:", train[train.History == 1].shape[0])
# print("Music:", train[train.Music == 1].shape[0])
# print("Mystery ", train[train.Mystery == 1].shape[0])
# print("Romance", train[train.Romance == 1].shape[0])
# print("Science ", train[train['Science Fiction'] == 1].shape[0])
# print("TV: ", train[train['TV Movie'] == 1].shape[0])
# print("Thriller: ", train[train.Thriller == 1].shape[0])
# print("War:", train[train.War == 1].shape[0])
# print("Western: ", train[train.Western == 1].shape[0])

In [ ]:
train = pd.concat([train, genres], axis=1, sort=False)
genres = train.genres.str.get_dummies(sep=',')
genres.columns

In [ ]:
train[train['genres'].str.contains('TV')]

**Only one movie has genre 'TV', so I decided to remove this genre.** 

In [ ]:
train.loc[train['genres'].str.contains('TV'), 'genres'] = 'Adventure,Drama,Science Fiction'

**Compare the revenues between different genres throughout the years:**

In [ ]:
# calculate the total revenue and avg revenue for each genre in each year, in order to plot the animation plot:
# colnames: 'genre', 'year', 'avg_rev', 'total_rev', 'num_of_productions'
def get_genre_animation_df(dataset, years, genres):
    result =[]
    for year in years:
        df_by_year = dataset[dataset['release_year'] == year]
        for genre in genres:
            df_by_year_and_genre = df_by_year[df_by_year[genre] == 1]
            result.append([year, genre, df_by_year_and_genre['revenue'].mean(), 
                           df_by_year_and_genre['revenue'].sum(),df_by_year_and_genre.shape[0]])
            
    return result
         

In [ ]:
df = get_genre_animation_df(train, np.arange(1912,2018), genres.columns)
genre_animation_plot_df = pd.DataFrame(df, columns=['year', 'genre', 'avg_rev', 'total_rev', 'num_of_productions'])
genre_animation_plot_df = genre_animation_plot_df.fillna(0)
px.scatter(genre_animation_plot_df, x='avg_rev', y = 'total_rev', animation_frame = 'year',
          size = 'num_of_productions', color='genre', hover_name = 'genre',
          size_max=55, range_x = [-3e+8, 7e+08], range_y = [-3e+8, 9e+09])

In [ ]:
# Here we count movies multiple times if it has multiple genres
def get_unpacked_df_by_genre(dataset, genres):
    result = pd.DataFrame(columns=['title', 'genre', 'revenue','budget'])
    for genre in genres:
        genre_df = dataset[dataset[genre] == 1][['title', 'genres', 'revenue', 'budget']]
        genre_df['genres'] = genre
        genre_df.columns = ['title', 'genre', 'revenue', 'budget']
        result = pd.concat([result, genre_df], ignore_index=True)
    return result

df_by_genre = get_unpacked_df_by_genre(train, genres.columns)
px.box(df_by_genre, x= 'genre', y='revenue')

    

**As it can be seen from both the animation plot and boxplot, genres like Advenure/Animation/Action/Fantasy are likely to make more revenues, as expected.**

### Language:

#### Exploring language related variables

In [ ]:
#count
[train['original_language'].count(), train['spoken_languages'].count()]

In [ ]:
#NA count
[(train['original_language'].isna()).any(),(train['spoken_languages'].isna()).sum()]

In [ ]:
#NaN columns of spoken_languages variable

#train['spoken_languages'].index('')
#print(train[train["spoken_languages"].isnull()][null_columns])
#NaN in train['spoken_languages']
train[train['spoken_languages'].isnull()]

In [ ]:
train['original_language'].head(15)

In [ ]:
train['spoken_languages'].head(15)

In [ ]:
#engineering spoken_languages column

#train['spoken_languages'].map(lambda x: sorted([d['iso_639_1'] for d in get_dictionary(x)])).map(lambda x: ','.join(map(str,x)))
train['spoken_languages_two'] = train['spoken_languages'].map(lambda x: sorted([d['iso_639_1'] for d in get_dictionary(x)]))

In [ ]:
train['spoken_languages_two'].head(15)
train['spoken_languages_two'][3]
train['spoken_languages_two'][3][1]
train['spoken_languages_two'][150] #empty for original NaN

In [ ]:
train['spoken_languages_two'].isna().sum() #0
train['spoken_languages_two'].count() #3000

In [ ]:
train['original_language'][1] in train['spoken_languages_two'][1]
train['orig_in_spok_language'] = np.array(0)

#for i in train['original_language']:
 #   if (train['original_language'][i] in train['spoken_languages_two'][i] == TRUE):
  #      train['orig_in_spok_language'] = 1
   # else:
    #    train['orig_in_spok_language'] = 0

In [ ]:
index_list = []
for idx, row in train.iterrows():
    index_list.append(row['original_language'] in row['spoken_languages_two'])

index_list #True means the original language is contained in spoken languages
len(index_list)

In [ ]:
not_index_list = [not i for i in index_list]
not_index_list #reverse of index_list

train[not_index_list] #those whose original language is not a part of spoken languages
len(train[not_index_list].index)

In [ ]:
#index_list.index(False) #92: first row of the one that is false
sum(not_index_list)
index_list.count(False)

#### Distribution of movies according to original languages usage

In [ ]:
train['original_language'].unique()

In [ ]:
len(train['original_language'].unique())

In [ ]:
#plt.figure(figsize=(20,15))
#sns.countplot(train['original_language'].sort_values())#alphabetical order
#plt.ylabel('Number of Movies')
#plt.xlabel('Original Languages')
#plt.title("Counts for Original Languages of Movies (alphabetical order)",fontsize=20)
#plt.show()

In [ ]:
train.groupby(['original_language']).original_language.count().nlargest(36) #top 5 of all 36 languages

In [ ]:
#plt.figure(figsize=(20,15))
#train_OrigLang=train.groupby(['original_language']).original_language.count().nlargest(36)
#bar1=sns.barplot(train_OrigLang.index, train_OrigLang.values,alpha=0.8)

#plt.ylabel('Number of Movies')
#plt.xlabel('Original Languages')
#plt.title("Counts for Original Languages of Movies (descending order)",fontsize=20)
#plt.show()

In [ ]:
#y_pos = train['original_language']
#performance = train['revenue']

#plt.bar(y_pos, performance, align='center', alpha=0.5)
#plt.ylabel('Revenue')
#plt.title('Revenue of Movies with Different Original Languages')

#plt.show()

In [ ]:
train_RevByOrigLang=train.groupby(['original_language']).revenue.mean()
train_RevByOrigLang.nlargest(5) 

In [ ]:
train_RevByOrigLang=train.groupby(['original_language']).revenue.mean()
train_RevByOrigLang=train_RevByOrigLang.nlargest(36) #descending order, 36 original languages in total

#plt.figure(figsize=(13,5))
bar2=sns.barplot(train_RevByOrigLang.index, train_RevByOrigLang.values,alpha=0.8)
#bar2.set_ylim([0,100000000])
plt.ylabel("Mean Revenue")
plt.title("Mean Revenue of Movies with Different Original Languages")

### Cast/Crew

In [ ]:
# TODO: 1. scrape data online to fill in the nan
#       2. use average revenue of crew/cast instead of total revenue

# skip the rows with no cast/crew info now, substitue with average revenue
## construct a list of dictionaries that records the average revenue of each crew/cast member:

dataset = train[pd.notnull(train['cast']) & pd.notnull(train['crew'])]
dataset['cast'] = dataset['cast'].apply(literal_eval)
dataset['crew'] = dataset['crew'].apply(literal_eval)

# cast:
rev_dict_list = {}
for idx, row in dataset.iterrows():
    for actor in row['cast']:
        if actor['name'] in rev_dict_list:
            rev_dict_list[actor['name']] = rev_dict_list[actor['name']] + row['revenue']
        else:
            rev_dict_list[actor['name']] = row['revenue']

for idx, row in dataset.iterrows():
    for actor in row['crew']:
        if actor['name'] in rev_dict_list:
            rev_dict_list[actor['name']] = rev_dict_list[actor['name']] + row['revenue']
        else:
            rev_dict_list[actor['name']] = row['revenue']

In [ ]:
# calculate accumulated revenue based on all the participants that appear in the dictionary:
dataset['total_personnel_rev'] = dataset['cast'].map(lambda x: (rev_dict_list[d['name']] for d in x)).map(sum) + \
                    dataset['crew'].map(lambda x: (rev_dict_list[d['name']] for d in x)).map(sum)



### Summary for release_year and release_month:
1. Revenues are generally low during Jan to Feb, also Aug to Sept. There are called Dump Months, more information can be found at:
    - https://www.wikiwand.com/en/Dump_months
    - https://web.archive.org/web/20151012234912/http://www.theatlantic.com/entertainment/archive/2012/01/january-dumping-ground-for-terrible-movies-like-contraband/251326/
    - https://web.archive.org/web/20150922062108/http://www.avclub.com/article/hollywoods-trash-our-treasure-17-salvageable-flops-91570
    

In [ ]:
train['spoken_languages'].head(15)

In [ ]:
country = {'Country': 'China', 'Capital': 'Beijing'}

### Production Company

In [ ]:
train['production_companies'].head(10)
train['production_companies'][8]

In [ ]:
train['production_companies'].isnull().sum()

In [ ]:
# TODO: use external datasets to fill in the nan in the production_company column
train_omit_prod_comp_nan = train[pd.notnull(train['production_companies'])] 
train_omit_prod_comp_nan['production_companies'] = train_omit_prod_comp_nan['production_companies'].apply(literal_eval)
train_omit_prod_comp_nan['production_companies'].head()

In [ ]:
len(train_omit_prod_comp_nan['production_companies'][0])

#see how many production companies are there for each movie
prod_comp_list = []
for idx, row in train_omit_prod_comp_nan.iterrows():
    prod_comp_list.append(len(row['production_companies']))

#number of prod companies for the movie with the most production companies
max(prod_comp_list)

#### New variable: number of production companies for each movie

In [ ]:
#add the variable showing the number of production companies for each movie
train_omit_prod_comp_nan['production_companies_num']=train_omit_prod_comp_nan['production_companies'].apply(len)
train_omit_prod_comp_nan.head(8)

In [ ]:
x = train_omit_prod_comp_nan['production_companies_num']
y = train_omit_prod_comp_nan['revenue']

#plt.scatter(x, y, s=area, c=colors, alpha=0.5)
plt.scatter(x, y, alpha=0.5)
plt.title('Revenue by Number of Production Companies')
plt.xlabel('Number of Production Companies')
plt.ylabel('Revenue')
plt.show()


In [ ]:
#creating the dataframe of x and y
x = train_omit_prod_comp_nan['production_companies_num']
y = train_omit_prod_comp_nan['revenue']
df_prod_comp_rev = pd.DataFrame(data=[x,y])
df_prod_comp_rev = df_prod_comp_rev.T #transposesns.pairplot(df_prod_comp_rev)
df_prod_comp_rev.head()
# import seaborn as sns
#correlation matrix: useless
# f, ax = plt.subplots(figsize=(10, 8))
# corr = df_prod_comp_rev.corr()
# sns.heatmap(corr, mask=np.zeros_like(corr, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
#             square=True, ax=ax)

In [ ]:
# boxplot of Revenue by Number of Production Companies

# library
import seaborn as sns, numpy as np
 
ax = sns.boxplot(x=df_prod_comp_rev["production_companies_num"], y=df_prod_comp_rev["revenue"], linewidth=2 )
ax = sns.stripplot(x='production_companies_num', y='revenue', data=df_prod_comp_rev, color="orange", jitter=0.3, size=2.5)

# Calculate number of obs per group & median to position labels
medians = df_prod_comp_rev.groupby(['production_companies_num'])['revenue'].median().values
# medians = [str(x) for x in medians.tolist()]
# medians = ["median: " + i for i in medians]
nobs = df_prod_comp_rev['production_companies_num'].value_counts().values
nobs = [str(x) for x in nobs.tolist()]
nobs = ["n = " + i for i in nobs]
 
# Add it to the plot
pos = range(len(nobs))
for tick,label in zip(pos,ax.get_xticklabels()):
    ax.text(pos[tick], medians[tick] + 0.03, nobs[tick], horizontalalignment='center', size='small', color='w', weight='semibold')

In [ ]:
train.columns

In [ ]:
df_prod_comp_rev.corr()

#### New metrics:

### Categorical Variables:

#### Original Language: